In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas
data_frame = pandas.read_csv('/kaggle/input/covid19-tweets/covid19_tweets.csv')
print(data_frame.shape)
data_frame.head()

In [ ]:
print('unique values for user_location field {}'.format(len(data_frame['user_location'].unique())))
print('null values for user_location field {}'.format(data_frame.isna().sum()['user_location']))
print('deleting rows with null values....')
data_frame.dropna(subset=['user_location'], inplace=True)
print(data_frame.shape)

In [ ]:
pip install geonamescache

In [ ]:
import geonamescache
gc = geonamescache.GeonamesCache()
country_names = gc.get_countries_by_names()
countries = gc.get_countries()
def get_country(value):
    if not isinstance(value, str):
        print(type(value))
        return value
    value = value.title()
    if country_names.get(value):
        return country_names[value]['name']
    if gc.get_cities_by_name(value):
        record = gc.get_cities_by_name(value)[0]
        return countries[record[next(iter(record.keys()))]['countrycode']]['name']
    split_values = value.split(' ')
    split_values = list(map(lambda x: x.title(), split_values))
    res = list(filter(lambda x: country_names.get(x), split_values))
    if res:
        return res[0]
    res = list(filter(lambda x: gc.get_cities_by_name(x), split_values))
    if res:
        record = gc.get_cities_by_name(res[0])[0]
        return countries[record[next(iter(record.keys()))]['countrycode']]['name']
    return 'Not found'

def boo(value):
    return value.lower()


In [ ]:
data_frame.user_location = data_frame.user_location.transform(get_country)
print(data_frame.user_location.value_counts())
print(data_frame.shape)
data_frame = data_frame[data_frame.user_location != 'Not found']
print(data_frame.shape)

In [ ]:
data_frame.user_location.value_counts()[:15].plot(kind='bar')